In [1]:
import os
import pandas as pd
%matplotlib inline

In [3]:
direct ='data/acs_1_yr_vet_status_est/'
ext = '.csv'

acs_dict = {
    "2005" : "data/acs_1_yr_vet_status_est/vets_2005_acs.csv",
    "2006" : "data/acs_1_yr_vet_status_est/vets_2006_acs.csv",
    "2007" : "data/acs_1_yr_vet_status_est/vets_2007_acs.csv",
    "2008" : "data/acs_1_yr_vet_status_est/vets_2008_acs.csv",
    "2009" : "data/acs_1_yr_vet_status_est/vets_2009_acs.csv",
    "2010" : "data/acs_1_yr_vet_status_est/vets_2010_acs.csv",
    "2011" : "data/acs_1_yr_vet_status_est/vets_2011_acs.csv",
    "2012" : "data/acs_1_yr_vet_status_est/vets_2012_acs.csv",
    "2013" : "data/acs_1_yr_vet_status_est/vets_2013_acs.csv",
    "2014" : "data/acs_1_yr_vet_status_est/vets_2014_acs.csv",
    "2015" : "data/acs_1_yr_vet_status_est/vets_2015_acs.csv",
    "2016" : "data/acs_1_yr_vet_status_est/vets_2016_acs.csv",
}

In [4]:
for i in acs_dict:
    print(i, acs_dict[i])

2005 data/acs_1_yr_vet_status_est/vets_2005_acs.csv
2006 data/acs_1_yr_vet_status_est/vets_2006_acs.csv
2007 data/acs_1_yr_vet_status_est/vets_2007_acs.csv
2008 data/acs_1_yr_vet_status_est/vets_2008_acs.csv
2009 data/acs_1_yr_vet_status_est/vets_2009_acs.csv
2010 data/acs_1_yr_vet_status_est/vets_2010_acs.csv
2011 data/acs_1_yr_vet_status_est/vets_2011_acs.csv
2012 data/acs_1_yr_vet_status_est/vets_2012_acs.csv
2013 data/acs_1_yr_vet_status_est/vets_2013_acs.csv
2014 data/acs_1_yr_vet_status_est/vets_2014_acs.csv
2015 data/acs_1_yr_vet_status_est/vets_2015_acs.csv
2016 data/acs_1_yr_vet_status_est/vets_2016_acs.csv
